In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.\
    	builder.\
    	appName("lab 2").\
    	master("spark://spark-master:7077").\
    	config("spark.executor.memory", "1000m").\
    	config("spark.executor.cores", "1").\
    	config("spark.cores.max", "4").\
    	getOrCreate()

23/06/13 03:09:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
spark

In [7]:
from pyspark.sql.functions import *

# Define a dataset.

df = spark.sparkContext.parallelize([
    (10, '', 10000), (20, 'Female', 30000), (None, 'Male', 80000), (None, 'Male', 5000)
]).toDF(["age", "gender", "income"])

df.show()


+----+------+------+
| age|gender|income|
+----+------+------+
|  10|      | 10000|
|  20|Female| 30000|
|null|  Male| 80000|
|null|  Male|  5000|
+----+------+------+



In [8]:
# Treat Null Value (None) with Average one.

avg_age = df.na.drop().agg(avg("age")).collect()[0][0]

sparkf_replaceNull = udf(lambda x: avg_age if x == None else x)

no_null_df = df.withColumn('age', sparkf_replaceNull(col('age')))

no_null_df.show()


+----+------+------+
| age|gender|income|
+----+------+------+
|  10|      | 10000|
|  20|Female| 30000|
|15.0|  Male| 80000|
|15.0|  Male|  5000|
+----+------+------+



In [10]:
df.na.drop().agg(avg("age")).collect()

[Row(avg(age)=15.0)]

In [13]:
type(df.na.drop().agg(avg("age")).collect()[0][0])

float

In [14]:
df.na.drop().agg(avg("age")).collect()[0][0]

15.0

In [17]:
df.withColumn('age',when(col('age').isNull(),avg_age).otherwise(col('age'))).show()

+----+------+------+
| age|gender|income|
+----+------+------+
|10.0|      | 10000|
|20.0|Female| 30000|
|15.0|  Male| 80000|
|15.0|  Male|  5000|
+----+------+------+



In [20]:
df.withColumn('new_gender',when(col('gender')=='','Male_Assume').otherwise(col('gender'))).show()

+----+------+------+-----------+
| age|gender|income| new_gender|
+----+------+------+-----------+
|  10|      | 10000|Male_Assume|
|  20|Female| 30000|     Female|
|null|  Male| 80000|       Male|
|null|  Male|  5000|       Male|
+----+------+------+-----------+



In [19]:
# Treat Missing Value with Defined Values.

from pyspark.sql.functions import *

treat_missing = udf(lambda x: "Male_Assume" if x == "" else x)

no_missing_df = no_null_df.withColumn('new_gender',treat_missing(no_null_df.gender))

no_missing_df.show()


+----+------+------+-----------+
| age|gender|income| new_gender|
+----+------+------+-----------+
|  10|      | 10000|Male_Assume|
|  20|Female| 30000|     Female|
|15.0|  Male| 80000|       Male|
|15.0|  Male|  5000|       Male|
+----+------+------+-----------+

